In [22]:
import pandas as pd
import pickle as pk
import numpy as np
import dgl
import torch as th

In [83]:
NUM_EPIC_NOUN_CLASS = 352
NUM_EPIC_VERB_CLASS = 125
KEY_EDGE_INTENSE = 'intense'

In [82]:

pkl = '/home/k1896871/Proj/gcn-master/gcn/EPIC_train_action_labels.pkl'
with open(pkl, 'rb') as f:
    meta = pk.load(f)

    # 39596 for the whole dataset
    # there are 28472 action segments for training
    # 2513 unique actions

    dic_participants_info = {}
    li_activities = []
    li_actions = []
    tmp_participant = 'P01'
    tmp_activity = 'P01_01'
    for index, data in meta.iterrows():
        v = data['verb_class']
        n = data['noun_class']
        vn = tuple((v,n))

        participant = data['participant_id']
        activity = data['video_id']

        if tmp_activity == activity:
            li_actions.append(vn)
            continue

        else:
            li_activities.append(li_actions)
            # dic_participant[activity[4:]] = li_actions
            tmp_activity = activity
            li_actions = []

        if tmp_participant != participant:

            dic_participants_info[tmp_participant] = li_activities
            li_activities = []
            tmp_participant = participant

In [81]:
p_keys = dic_participants_info.keys()
graphs = []
print(p_keys)
for p_id in p_keys:
    activity_list = dic_participants_info[p_id]
    
    for actions in activity_list:
      
        ns = np.array(actions)[:,1]

        print (ns)

dict_keys(['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P10', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30'])
[  8 113   8  10 185  29 159  10  10  40   9  10  90   8  19  19   8   9
   5   5   5   9   5   3  69  69  40  40   3  90   8 131   1   1   8   8
  69  69  47  69  69  69  69  69  69  69  69   9   9  22  69  31  31  31
  31  69  22 185 185   3 185   3 185 185 185 185  19 185 185  10 185  10
  22  22  22  29  13  13  13  29  22   5  13  13  13   5  38   3  17  17
   3  38  22  90  90  16  16   8  16   8  23   8  23  29  29  29  29  11
  29   4   4   9   9   9   9  18  18  22  90  29  10  29  10  22  90 131
 131  40  40 131  40  40  40  40 131  40 131 131  40  22  90   8   1   1
  47  38  17  17  38  24  24  24  24  22  19  13  19  90  90  31  31  31
   9   7  31  31  17  31   7  31  22  90  47  90  90  90  22   8   8   8
  44  44  44  22  90  90  22 159   5 159  29 159  29  90  22  29  29 

In [80]:
# initial Graph
G = dgl.DGLGraph(multigraph=True)
num_node = ns.size
G.add_nodes(NUM_EPIC_NOUN_CLASS,{'x':th.zeros((NUM_EPIC_NOUN_CLASS,5))})
print(G.ndata)
u = ns[0]
# print(ns.size) 326 actions in P0101 so 175 links should be fine, as there may be only 175 unique nodes

# next_ns = np.zeros(ns.size-1, dtype=np.int32)
# next_ns[0:] = ns[1:]
# print(next_ns)
# G.add_edges(ns[:-1],next_ns,{'intense':th.ones(num_node-1)})
for i in range(1,num_node):
    v = ns[i]
#     print(u,v)
    if not G.has_edge_between(u,v):
       
        G.add_edges(u,v,{KEY_EDGE_INTENS:th.ones([1])})
    else:
        
        G.edges[u,v].data[KEY_EDGE_INTENS] += 1 
        
    u = v
print(G.edata)
    


{'x': tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])}
{'intense': tensor([ 1.,  1.,  1.,  2.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  3.,  1.,
         1.,  1.,  1.,  2.,  2.,  1.,  1.,  1.,  9.,  1.,  5.,  1.,  1.,  1.,
         1.,  4.,  1.,  3.,  1.,  1.,  1.,  1.,  4.,  3.,  1.,  1.,  6.,  1.,
         1.,  1.,  5.,  2.,  2.,  1.,  1.,  2.,  2.,  6.,  3.,  1.,  4.,  1.,
         5.,  1.,  1.,  1.,  1.,  1.,  1.,  3.,  1.,  1.,  1., 13.,  9.,  1.,
         3.,  2.,  1.,  2.,  1.,  1., 10.,  1.,  1.,  1.,  3.,  1.,  1.,  1.,
         1.,  1.,  2.,  1.,  1.,  2.,  4.,  3.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  3.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,
         1.,  1.,  1.,  8.,  1.,  2., 20.,  6.,  1.,  1.,  1.,  2.,  1.,  3.,
         1.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,
       